In [11]:
from datasets import load_dataset

name_to_push_dataset_to = "cubbk/rixvox-tokenised"


df = load_dataset("parquet", data_files="../dataset/dev_metadata.parquet")["train"] # type: ignore

df = df.remove_columns(["dokid", "anforande_nummer", "observation_nr", "party", "gender", "debatedate", "electoral_district", "birth_year", "intressent_id", "speaker_from_id", "speaker_audio_meta", "start", "end", "duration", "bleu_score", "speaker_total_hours"])



In [12]:
import os
import numpy as np
import soundfile as sf
from datasets import Dataset

# Build path column
ds = df.map(lambda ex: {"audio_path": os.path.join("../dataset/dev/", ex["filename"])}) # type: ignore

def load_audio(example):
    y, sr = sf.read(example["audio_path"])
    if y.ndim > 1:
        y = np.mean(y, axis=1)  # mono
    example["waveform"] = y.astype("float32")
    example["sampling_rate"] = sr
    return example

ds = ds.map(load_audio) # type: ignore
ds = ds.remove_columns(["filename", "audio_path"])
ds

Map:   0%|          | 0/7933 [00:00<?, ? examples/s]

Map:   0%|          | 0/7933 [00:00<?, ? examples/s]

Dataset({
    features: ['speaker', 'text', 'waveform', 'sampling_rate'],
    num_rows: 7933
})

In [14]:
ds.to_parquet("../dataset_parquet/dev.parquet")

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

11932738560